# Testing reverse correlation

Taken from [https://doi.org/10.1016/j.jneumeth.2021.109297](https://doi.org/10.1016/j.jneumeth.2021.109297)

In [ ]:
# import librairies

import struct
import time

import numpy as np
import scipy.stats as stt

from sklearn.model_selection import StratifiedShuffleSplit

# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# PCA
from sklearn.decomposition import PCA

from numba import jit

import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb

font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 18}
mpl.rc('font', **font)

Load the MNIST dataset: images and labels

In [ ]:
data_dir = './'

# use test set as data here
with open(data_dir+'t10k-images-idx3-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    data = data.reshape((size, nrows, ncols))

with open(data_dir+'t10k-labels-idx1-ubyte','rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    label = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))


data = np.array(data, dtype=float) / 255
label = np.array(label, dtype=int)

print('array shapes:', data.shape, label.shape)

# number of samples
n_smpl = data.shape[0]
n_px = data.shape[1]
n_feat = n_px**2

# format in scikit learn format
X = data.reshape([n_smpl,-1])
y = label

# number of classes
n_cl = np.unique(y).size

In [ ]:
# plot example
plt.figure()
plt.imshow(data[0], cmap='binary')

We want to compare the accuracy and reverse image of different classifiers:
- MLR is the multinomial logistic regression
- 1NN is the 1-nearest-neighbor
- RF is a random forest

In [ ]:
# classifiers

clf = LogisticRegression(penalty='l2', C=0.1, max_iter=1000)

# clf = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='minkowski')

# clf = RandomForestClassifier(n_estimators=10, criterion='gini')

Usual cross-validation scheme

In [ ]:
# 80% for training and 20% for testing
cvs = StratifiedShuffleSplit(n_splits=20, test_size=0.2)

In [ ]:
# dataframe to store result accuracies
df_acc = pd.DataFrame()

# repeat classification
for train_ind, test_ind in cvs.split(X, y):
    # fit to train data
    clf.fit(X[train_ind,:], y[train_ind])
    # test accuracy
    d = {'type': ['test'],
         'score': [clf.score(X[test_ind,:], y[test_ind])]}
    df_acc = pd.concat((df_acc, pd.DataFrame(data=d)), ignore_index=True)

    # shuffling labels and fit again for evaluation of chance level
    train_ind_rand = np.random.permutation(train_ind)
    clf.fit(X[train_ind,:], y[train_ind_rand])
    d = {'type': ['shuf'],
         'score': [clf.score(X[test_ind,:], y[test_ind])]}
    df_acc = pd.concat((df_acc, pd.DataFrame(data=d)), ignore_index=True)
        
# print table of results
print(df_acc)

In [ ]:
# test versus shuffle accuracy
sb.violinplot(data=df_acc, x='type', y='score', order=['shuf','test'],
              density_norm='width', palette=['gray','orange'])
plt.plot([-1,2], [1.0/n_cl]*2, '--k')
plt.yticks([0,1])
plt.ylabel('accuracy')
plt.show()

In [ ]:
# input PCA space
n_comp = 10
pca = PCA(n_components=n_comp)
pca.fit(X)

In [ ]:
# train classifier
clf.fit(X, y)

In [ ]:
sig_bub = 2.0
prec_bub = np.eye(2) / sig_bub**2
det_prec_bub_2pi = 1.0 / 2 / np.pi / sig_bub**2

def gaussian_2d(x, mean, prec, det_prec_2pi):
    # assert means.shape==(2,) and covs.shape==(2,2)
    return np.exp(-np.einsum('kli,ij,klj->kl', x-mean, prec, x-mean) / 2.0) * det_prec_2pi

def gen_bubbles(n_bub=10):
    x, y = np.meshgrid(np.arange(n_px), np.arange(n_px))
    pos = np.dstack((x, y))
    means = np.random.randint(n_px, size=[n_bub,2])
    xy = np.zeros([n_px,n_px])
    for i_bub in range(n_bub):
        xy += gaussian_2d(pos, means[i_bub,:], prec_bub, det_prec_bub_2pi)
    return xy

In [ ]:
plt.imshow(gen_bubbles().reshape([n_px,n_px]))
plt.show()

In [ ]:
# number of noisy samples
n_ns = 10000

# noisy inputs
method = 'raw' # raw noise, pca noise, bubble, alone or combined with stimulus
if method=='raw':
    # noise in original image space
    X_ns = np.random.normal(loc=0.0, scale=1.0, size=[n_ns,n_feat])
elif method=='pca':
    # noise in PCA space
    X_ns = pca.inverse_transform(np.random.normal(loc=0.0, scale=2.0, size=[n_ns,n_comp]))
elif method=='pca+stim':
    # additive noise in PCA space on top of stimulus
    X_ns = X[np.random.randint(n_smpl, size=[n_ns]),:]
    X_ns += pca.inverse_transform(np.random.normal(loc=0.0, scale=0.5, size=[n_ns,n_comp]))
elif method=='bub':
    # bubbles are Gaussian randomly distributed in original space (2D image)
    bub_masks = np.zeros([n_ns,n_px,n_px])
    for i_ns in range(n_ns):
        bub_masks[i_ns,:,:] = gen_bubbles()
    bub_masks = bub_masks.reshape([n_ns,n_px**2])
    X_ns = bub_masks * 50.0
elif method=='bub*stim':
    # mulitplicative noise bubble
    X_ns = X[np.random.randint(n_smpl, size=[n_ns]),:]
    bub_masks = np.zeros([n_ns,n_px,n_px])
    for i_ns in range(n_ns):
        bub_masks[i_ns,:,:] = gen_bubbles()
    bub_masks = bub_masks.reshape([n_ns,n_px**2])
    X_ns *= bub_masks * 100.0
else:
    raise ValueError('unknown method')

# prediction from trained classifier
y_pred = clf.predict(X_ns)

In [ ]:
# check correct prediction for digit 0
i_cl = 3
y_correct = np.array(y_pred==i_cl, dtype=int)
print('ratio correct predicitons:', y_correct.sum()/n_ns)

# correlation between prediction for digit 0 and inputs
X_y_1 = np.einsum('si,s->i', X_ns, y_correct) # 1st moment
X_y_2 = np.einsum('si,s->i', X_ns**2, y_correct) # 2nd moment
X_corr = ( X_y_2 - X_y_1**2 ) / n_ns # covariance
X_corr /= np.std(X_ns, axis=0) * np.std(y_correct)

#  plot "reverse image"
data_inv = -X_corr.reshape([n_px,n_px])
plt.figure()
plt.imshow(data_inv, cmap='binary')
plt.colorbar()
plt.show()